In [ ]:
import geopandas as gpd
import rasterio
import pandas as pd
from rasterio.sample import sample_gen
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,ConfusionMatrixDisplay,\
                            precision_score,recall_score,f1_score,roc_auc_score, roc_curve

# import matplotlib as plt
import matplotlib.pyplot as plt

from lightgbm import LGBMClassifier  #https://github.com/sentinel-hub/eo-learn-examples/blob/main/custom-script/machine-learning-evalscript.ipynb
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
# Load the CSV data
# data = pd.read_csv('training_data_python_textures.csv')
training_data_path="output/sample/training_data_eaton_indices.csv"
training_data=pd.read_csv(training_data_path)

# get from feature_extration.ipynb
# columns_names=["gam_vh_post","gam_vv_post","gam_vh_pre","gam_vv_pre"]
# columns_names=["gam_vh_post","gam_vv_post","gam_vh_pre","gam_vv_pre","RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH", "ΔRVI"]
columns_names= feature_column_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI",'Δvv_vh_ratio',"ΔRDFI","RCBI",'t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20'] #,"ΔRVI",'Δvv_vh_ratio',"ΔRDFI","dTRAD"


# columns_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH", "ΔRVI"]
# columns_names=['t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20']

# columns_names=["gamm_vh_pre","gamm_vv_pre","gamm_vh_post","gamm_vv_post","RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH", "ΔRVI",'t1','t2','t3','t4','t5','t6','t7','t8','t9','t10','t11','t12','t13','t14','t15','t16','t17','t18','t19','t20']


In [ ]:
# Separate features (SAR bands) and labels (e.g., class column)
X = training_data[columns_names]  # Features (SAR band values)
Y = training_data['class']  # Labels (add 'Class' column in your CSV if available)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


In [ ]:
#To visualize data in X_train
pd.DataFrame(X_train)

,RBD_VV,RBD_VH,RBR_VV,RBR_VH,ΔRVI,Δvv_vh_ratio,ΔRDFI,RCBI,t1,t2,...,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20
6590,-0.087255,0.021363,0.616283,1.736909,0.605012,-5.061173,-0.302507,-0.065892,90.975050,11.357450,...,162.661990,13.140408,0.110643,-0.054694,-6.422003,-52.228573,-1438.2506,51.456444,1341.4714,-9.199298
4022,0.010978,0.040024,1.082248,2.552171,0.604354,-2.981064,-0.302175,0.051003,156.038620,14.580408,...,-0.864286,-0.938673,0.319193,0.510102,0.605211,-60.378570,-1894.7244,60.564484,1866.4785,-4.828771
5571,0.022116,0.006646,1.297951,1.238012,-0.037138,0.128667,0.018571,0.028763,61.133110,8.190919,...,99.127780,10.930000,0.110479,-0.051429,-6.877317,-53.514286,-1473.1967,52.683617,1397.5071,-7.176474
3308,0.000643,-0.007890,1.013868,0.481734,-0.449207,3.359316,0.224601,-0.007247,45.255054,7.420204,...,40.933113,7.492755,0.157336,0.000102,-7.511490,-49.121430,-1219.8360,48.188255,1190.1714,-5.673970
2117,0.096656,-0.005714,1.774411,0.823858,-0.394492,4.437782,0.197249,0.090942,107.540344,12.009082,...,3.196302,1.696225,0.359791,0.018061,-2.541526,-59.928570,-1825.3289,59.444027,1710.9714,-11.829743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5226,0.055692,-0.011859,1.285346,0.700871,-0.276479,4.104625,0.138240,0.043833,49.681892,8.035001,...,2.388571,1.715816,0.488950,0.098469,-3.063969,-58.928574,-1750.2422,58.200993,1730.4857,-4.429565
5390,-0.063306,-0.008790,0.765181,0.806306,0.026318,-0.303096,-0.013160,-0.072096,103.928024,10.727654,...,0.022857,0.448469,0.522480,0.237245,-1.283379,-60.114285,-1838.2048,59.657530,1706.4286,-12.000388
860,0.047628,0.106235,1.191413,4.719411,0.838434,-6.512275,-0.419216,0.153863,83.971700,10.533571,...,7.909406,2.734286,0.261567,-0.048571,-3.031769,-59.521427,-1801.1464,59.023876,1776.2072,-4.731021
7603,-0.013742,0.004535,0.896685,1.123438,0.162622,-0.731040,-0.081311,-0.009207,75.338440,9.677245,...,10.951003,3.188367,0.387974,0.025918,-4.307225,-58.521427,-1724.7246,57.724842,1667.9358,-8.031986


## To choose the models for hyper parameter tuning

In [ ]:
models={
    "Random Forest":RandomForestClassifier(),
    "Decision Tree":DecisionTreeClassifier(),
    # "Logistic Regression":LogisticRegression(), #NOT GOOD
    # "GradientBoost Classifier": GradientBoostingClassifier(), #NOT GOOD
    "Light GBM":LGBMClassifier(),
    "XGB Classifier":XGBClassifier()
}

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train, y_train)

    #Make a prediction
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)

    #Training set performance
    model_train_accuracy=accuracy_score(y_train,y_train_pred)
    model_train_f1=f1_score(y_train,y_train_pred,average='weighted')
    model_train_precision=precision_score(y_train,y_train_pred)
    model_train_recall=recall_score(y_train,y_train_pred)
    model_train_roc_auc_score=roc_auc_score(y_train,y_train_pred)

    #Test set performance
    model_test_accuracy=accuracy_score(y_test,y_test_pred)
    model_test_f1=f1_score(y_test,y_test_pred,average='weighted')
    model_test_precision=precision_score(y_test,y_test_pred)
    model_test_recall=recall_score(y_test,y_test_pred)
    model_test_roc_auc_score=roc_auc_score(y_test,y_test_pred)

    # print(model)
    print('Model performance for Training set')
    print(f"- Accuracy: {model_train_accuracy}")
    print(f"- F1 Score: {model_train_f1}")
    print(f"- Precision Score: {model_train_precision}")
    print(f"- Recall Score: {model_train_recall}")
    print(f"- Roc Auc Score: {model_train_roc_auc_score}")

    print("--------------")
    print('Model performance for Test set')
    print(f"- Accuracy: {model_test_accuracy}")
    print(f"- F1 Score: {model_test_f1}")
    print(f"- Precision Score: {model_test_precision}")
    print(f"- Recall Score: {model_test_recall}")
    print(f"- Roc Auc Score: {model_test_roc_auc_score}")
    print("----------------------------------------")


# randomforest and decisiontrees will be option for further hyperparameter tuning

Model performance for Training set
- Accuracy: 1.0
- F1 Score: 1.0
- Precision Score: 1.0
- Recall Score: 1.0
- Roc Auc Score: 1.0
--------------
Model performance for Test set
- Accuracy: 0.751727115716753
- F1 Score: 0.7221743067319851
- Precision Score: 0.6734104046242775
- Recall Score: 0.33525179856115106
- Roc Auc Score: 0.6327708715199339
----------------------------------------
Model performance for Training set
- Accuracy: 1.0
- F1 Score: 1.0
- Precision Score: 1.0
- Recall Score: 1.0
- Roc Auc Score: 1.0
--------------
Model performance for Test set
- Accuracy: 0.6636442141623489
- F1 Score: 0.661096359809772
- Precision Score: 0.43636363636363634
- Recall Score: 0.4143884892086331
- Roc Auc Score: 0.5924502594099921
----------------------------------------
[LightGBM] [Info] Number of positive: 1617, number of negative: 3787
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007813 seconds.
You can set `force_col_wise=true` to remove the o

In [ ]:
#Hyperparameter training
rf_params={"max_depth":[5,8,15,12,10],
           "max_features":[5,7,1,2,3,4,9,12,20,30,40],
           "min_samples_split":[2,8,15,20,30,40],
           "n_estimators":[100,150,200,300]
           }

#best param {'n_estimators': 50, 'max_depth': 20, 'learning_rate': 0.1}
# bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
xgboost_params={"max_depth":[10,20,30,40],
        #    "max_features":[5,7,8,9],
           "learning_rate":[0.001,0.01,0.5,0.1,1],
           "n_estimators":[32,50,100,200]
           }
#models list for hyperparameter tuning

randomcv_models=[
    ("RF",RandomForestClassifier(), rf_params),
    # ("XGBoost",XGBClassifier(), xgboost_params),

    # ("DT",DecisionTreeClassifier(), dt_params),
]


In [ ]:
model_param={}

for name, model, params in randomcv_models:
    random=RandomizedSearchCV(estimator=model,param_distributions=params,n_iter=100,cv=3,verbose=2,n_jobs=-1)
    random.fit(X_train,y_train)
    model_param[name]=random.best_params_

for model_name in model_param:
    print(f"best param for {model_name}-----")
    print(model_param[model_name])

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END max_depth=5, max_features=3, min_samples_split=20, n_estimators=200; total time=   1.9s
[CV] END max_depth=5, max_features=3, min_samples_split=20, n_estimators=200; total time=   2.0s
[CV] END max_depth=5, max_features=3, min_samples_split=20, n_estimators=200; total time=   2.0s
[CV] END max_depth=12, max_features=4, min_samples_split=30, n_estimators=150; total time=   3.2s
[CV] END max_depth=12, max_features=4, min_samples_split=30, n_estimators=150; total time=   3.2s
[CV] END max_depth=12, max_features=4, min_samples_split=30, n_estimators=150; total time=   3.3s
[CV] END max_depth=10, max_features=7, min_samples_split=2, n_estimators=150; total time=   5.1s
[CV] END max_depth=10, max_features=7, min_samples_split=2, n_estimators=150; total time=   5.3s
[CV] END max_depth=10, max_features=7, min_samples_split=2, n_estimators=150; total time=   5.2s
[CV] END max_depth=5, max_features=30, min_samples_split=40, 

In [ ]:
# https://www.datacamp.com/tutorial/random-forests-classifier-python
from scipy.stats import randint
param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

# Create a random forest classifier
rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

# Fit the random search object to the data
rand_search.fit(X_train, y_train)
# Create a variable for the best model
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

KeyboardInterrupt: 